In [1]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import time

main_dir = os.getcwd()
os.chdir(main_dir + '\\Leaps Pulling')
cad_dir = main_dir + '\\Historical Queries\\CAD'
usd_dir = main_dir + '\\Historical Queries\\US'

from helpers import *
from yahoo_query import *

os.chdir(main_dir)

In [209]:
def create_dgi(annual_data):
    dgi_data = annual_data[['totalLiab',
                             'totalStockholderEquity',
                             'longTermInvestments',
                             'shortTermInvestments',
                             'netIncome','operatingIncome',
                             'totalRevenue','dividendsPaid',
                             'investments','Underlying']]
    dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

    dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
    dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
    dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
    dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)

    dgi_data = dgi_data[['Underlying','payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
    return dgi_data

# DGI Portion

def dgi_scores(annual_df,keystats, min_dgi_score):
    dgi_df = create_dgi(annual_df).fillna(0).replace(-np.inf,0).replace(np.inf,0)

    dgi_df_scores = pd.DataFrame(columns = ['divGrowth','payoutChange','payoutRatio',
                                             'debtEquityRatio','netMargin','roic'],
                                  index = dgi_df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in dgi_df.Underlying.drop_duplicates():
        curr_dgi = dgi_df[dgi_df.Underlying == ticker].sort_index()
        if len(curr_dgi) - 1:
            if sum(curr_dgi['dividendsPaid'].pct_change() >= 0.02) == len(curr_dgi) -1:
                dgi_df_scores.loc[ticker, 'divGrowth'] = 1
            if sum(curr_dgi.payoutRatio.pct_change() <= 1) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'payoutChange'] = 1
            if curr_dgi.payoutRatio.tail(3).mean() <= 0.6:
                dgi_df_scores.loc[ticker, 'payoutRatio'] = 1
            if sum(curr_dgi.debtEquityRatio.tail(3) <= 1.5) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'debtEquityRatio'] = 1
            if sum(curr_dgi.netMargin.tail(3) >= 0.07) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'netMargin'] = 1
            if sum(curr_dgi.roic.tail(3) >= 0.2) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'roic'] = 1

    dgi_df_scores['score'] = dgi_df_scores.sum(axis = 1)
    dgi_df_scores = dgi_df_scores[dgi_df_scores.score >= min_dgi_score].join(keystats[['currentPrice','returnOnAssets',
                                                                                       'returnOnEquity','sector']])

    dgi_df_scores = dgi_df_scores.sort_values('returnOnAssets', ascending = False)
    dgi_df_scores['roaRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores = dgi_df_scores.sort_values('returnOnEquity', ascending = False)
    dgi_df_scores['roeRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores['roRank'] = dgi_df_scores.roaRank + dgi_df_scores.roeRank
    dgi_df_scores = dgi_df_scores.sort_values('roRank')
    return dgi_df_scores[['currentPrice','score','roRank','sector']]

# filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
#        (dgi_df['divGrowth'] >= 0.03) &
#        (dgi_df['payoutChange'] <= 0) &
#        (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
#        (dgi_df['netMarginAvg'] >= 0.07) &
#        (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)



# LEAPs Portion

def leaps_scores(data_df, keystats, min_score):
    try:
        df = data_df[['Underlying', 'earnings', 'netIncome','totalRevenue','totalStockholderEquity']]
    except:
        df = data_df[['Underlying', 'epsActual', 'netIncome','totalRevenue','totalStockholderEquity']]
    ks = keystats[['52WeekChange','currentPrice','sector','forwardPE','forwardEps','pegRatio']]
    ks['52WeekChange'] = pd.to_numeric(ks['52WeekChange'].replace('Infinity',0))

    df['profitMargin'] = df.netIncome/df.totalRevenue
    df['roe'] = df.netIncome/df.totalStockholderEquity

    # us_quarterly[['Underlying', 'retainedEarnings', 'netIncome','totalRevenue','totalStockholderEquity']]

    leap_scores = pd.DataFrame(columns = ['earningsGrowth','profitMargins','profitMarginChange',
                                          'roe','roeChange'],
                               index = df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in df.Underlying.drop_duplicates():
        curr_data = df[df.Underlying == ticker].sort_index()
        curr_len = len(curr_data) - 1
        if curr_len > 1:
            try:
                if sum(curr_data.earnings.pct_change() >= 0.05) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            except:
                if sum(curr_data.epsActual.pct_change() > 0) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            if sum(curr_data.profitMargin.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'profitMargins'] = 1
            if sum(curr_data.profitMargin.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'profitMarginChange'] = 1
            if sum(curr_data.roe.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'roe'] = 1
            if sum(curr_data.roe.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'roeChange'] = 1

    leap_scores['score'] = leap_scores.sum(axis = 1)
    leap_scores = leap_scores[leap_scores.score >= min_score].join(ks).sort_values('52WeekChange', ascending = False)
    return leap_scores


# DGI Filter

In [172]:
os.chdir(cad_dir)

file_date = '2018-09-18'
cad_annual = pd.read_csv('cad_annual_{}.csv'.format(file_date), index_col = 0)
cad_keystats = pd.read_csv('cad_keystats_{}.csv'.format(file_date), index_col = 0)
cad_quarterly = pd.read_csv('cad_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

os.chdir(usd_dir)

file_date = '2018-09-19'
us_annual = pd.read_csv('us_annual_{}.csv'.format(file_date), index_col = 0)
us_keystats = pd.read_csv('us_keystats_{}.csv'.format(file_date), index_col = 0)
us_quarterly = pd.read_csv('us_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

In [158]:
cad_dgi_scores = dgi_scores(cad_annual, cad_keystats, 4)
print("CAD")
cad_dgi_scores.head(20)

,currentPrice,score,roRank,sector
TCN.TO,11.17,4,8,Real Estate
BAD.TO,28.45,4,10,Industrials
WTE.TO,26.56,4,11,Basic Materials
CNR.TO,113.95,5,12,Industrials
LUC.TO,2.31,4,12,Basic Materials
MG.TO,69.86,4,20,Consumer Cyclical
ENGH.TO,80.15,5,21,Technology
TIH.TO,64.41,4,21,Industrials
MRU.TO,41.07,4,22,Consumer Defensive
RCH.TO,30.48,5,24,Consumer Cyclical


In [173]:
us_dgi_score = dgi_scores(us_annual, us_keystats, 5)
print("USD")
us_dgi_score.head(20)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

USD


,currentPrice,score,roRank,sector
MA,218.120,5,4,Financial Services
INTU,220.000,5,7,Technology
TXN,107.110,6,8,Technology
SBUX,55.470,5,12,Consumer Cyclical
AMAT,38.980,5,13,Technology
ACN,173.105,5,15,Technology
MKTX,183.210,5,19,Financial Services
ROL,62.450,5,22,Industrials
AAPL,217.700,5,24,Technology
SEIC,62.390,5,24,Financial Services


# LEAPs Filtering

In [ ]:
leaps_annual = leaps_scores(us_annual, us_keystats, 4)#.replace('Infinity', 0)
leaps_quarterly = leaps_scores(us_quarterly, us_keystats, 4)#.replace('Infinity', 0)


C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [ ]:
leaps_annual[(leaps_annual.profitMargins > 0) &
             (leaps_annual.earningsGrowth > 0) &
             (leaps_annual.roe > 0) &
             (leaps_annual.profitMarginChange > 0) &
             (pd.to_numeric(leaps_annual['52WeekChange']) > 0.1)]

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
os.chdir(main_dir + '\\Single Name Pulls')
datenow = dt.datetime.today().strftime("%Y-%m-%d")
filename = 'single_names {}.xlsx'.format(datenow)

writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
cad_dgi_scores.to_excel(writer, sheet_name='ca_dgi')
us_dgi_score.to_excel(writer, sheet_name='us_dgi')
leaps_annual.to_excel(writer, sheet_name='leaps_a')
leaps_quarterly.to_excel(writer, sheet_name = 'leaps_q')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
os.chdir(main_dir)